In [1]:
from pulp import *
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# Input data
heat_demand = pd.read_csv('heat_demand_48h.csv', index_col=False)
el_demand = pd.read_csv('electric_demand.csv', index_col=False)
el_prices = pd.read_csv('electric_prices.csv', index_col=False)
eon_elprice = pd.read_csv('eon_elprice.csv', index_col=False)


In [ ]:
heat_demand

In [4]:
# Input data
#heat_demand = pd.read_csv('heat_demand.csv', index_col=False)
#el_demand = pd.read_csv('electric_demand.csv', index_col=False)
#el_prices = pd.read_csv('electric_prices.csv', index_col=False)
#eon_elprice = pd.read_csv('eon_elprice.csv', index_col=False)

# Resource data
resources_heat = {
   
    'CHP_heat': {
        'min': 6,
        'max': 18.5,
        'efficiency': 1,
        'cost': 195.4       
         },
    'WC1_boiler': {
        'min': 4,
        'max': 9,
        'efficiency': 0.85,
        'cost': 195.4
    },
    'bio_oil_boiler_1': {
        'min': 0,
        'max': 6,
        'efficiency': 0.76,
        'cost': 1548.6
    },
    'bio_oil_boiler_2': {
        'min': 0,
        'max': 6,
        'efficiency': 0.76,
        'cost': 1548.6
    },
    'wood_pellets_boiler_1': {
        'min': 0.5,
        'max': 3.25,
        'efficiency': 0.87,
        'cost': 455.2 
    },
    'wood_pellets_boiler_2': {
        'min': 0.5,
        'max': 3.25,
        'efficiency': 0.87,
        'cost': 455.2 
    },
    'bio_oil_boiler_3': {
        'min': 0,
        'max': 10,
        'efficiency': 0.875,
        'cost': 1548.6
    }
}

resources_el = {
  'CHP_el': {
        'min': 0.8,
        'max': 3.9,
        'efficiency': 0.4,
        'cost': 248.5
    }}

Network_losses=0.091 # 9.1%

Storage_capacity = 100 #MWh PRELIMINARY VALUE

Storage_losses = 0.015 #1.5%/hour PRELIMINARY VALUE

Storage_maxrate = 30 #MW PRELIMINARY VALUE


# Initialize the optimization problem....................................................................................................

prob = LpProblem("District Heating Plant Dispatch Optimization", LpMinimize)


# Initialize the decision variables....................................................................................................

variables = LpVariable.dicts("Fuel", ((r, t) for r in resources_heat for t in heat_demand.index), lowBound=0, cat='Continuous')
b_heat = LpVariable("b_heat", cat="Binary")
storage_in=LpVariable.dict("Storage in", [t for t in heat_demand.index], lowBound=0, upBound=Storage_maxrate, cat='Continuous') # MWh
storage_out=LpVariable.dict("Storage out", [t for t in heat_demand.index], lowBound=0, upBound=Storage_maxrate, cat='Continuous') # MWh


      
# Objective function.............................................................................................

prob += lpSum([variables[(r, t)] * resources_heat[r]['cost'] for r in resources_heat for t in heat_demand.index])

Storage_state=0

# Constraints....................................................................................................
for t in heat_demand.index:
    #Charge/Discharge rate
    
    prob += storage_in[t]<= Storage_maxrate
    prob += storage_out[t]<= Storage_maxrate
    prob += storage_in[t]>= 0
    prob += storage_out[t]>= 0
    
    # Heat demand constraint
    heat_production = lpSum([variables[(r, t)]  for r in resources_heat]) 
                     
    
    prob += heat_production >= (1+Network_losses)*heat_demand.loc[t]  + storage_in[t] -  storage_out[t] 

    #Storage constrains
    
    Storage_state =(Storage_state + storage_in[t] - storage_out[t])*(1-Storage_losses)
                         
    prob += Storage_state <= Storage_capacity
    prob += Storage_state >= 0

    
    # Resource capacity constraints
    for r in resources_heat:
            prob += variables[(r, t)] <= resources_heat[r]['max']

    # Resource minimum load constraints
    prob += variables[(r, t)] >= resources_heat[r]['min']*b_heat
    
    

    
     
    
    # Solve the optimization problem
prob.solve()


# Print the status of the solution
print("Status:", LpStatus[prob.status])

# Print the optimal value of the objective function
#print("Optimal Value: ", value(prob.objective))

# Print the values of the decision variables
for v in prob.variables():
    print(v.name, "=", v.varValue)


# Define decision variables
chp_heat = {}
WC1 = {}
bio_oil_1 = {}
bio_oil_2 = {}
wood_pellets_1 = {}
wood_pellets_2 = {}

# Populate the decision variables with the optimal values
for i in heat_demand.index:
    chp_heat[i] = variables[('CHP_heat', i)]
    WC1[i] = variables[('WC1_boiler', i)]
    bio_oil_1[i] = variables[('bio_oil_boiler_1', i)]
    bio_oil_2[i] = variables[('bio_oil_boiler_2', i)]
    wood_pellets_1[i] = variables[('wood_pellets_boiler_1', i)]
    wood_pellets_2[i] = variables[('wood_pellets_boiler_2', i)]

# Populate the results DataFrame with the optimal values of the decision variables
results = pd.DataFrame(index=heat_demand.index, columns=['CHP_heat','WC1_boiler', 'Bio_oil_1', 'Bio_oil_2', 'wood_pellets_boiler_1', 'wood_pellets_boiler_2'])

for i in heat_demand.index:
    results.loc[i, 'CHP_heat'] = chp_heat[i].varValue
    results.loc[i, 'WC1_boiler'] = WC1[i].varValue
    results.loc[i, 'Bio_oil_1'] = bio_oil_1[i].varValue
    results.loc[i, 'Bio_oil_2'] = bio_oil_2[i].varValue
    results.loc[i, 'Bio_oil_3'] = bio_oil_2[i].varValue
    results.loc[i, 'wood_pellets_boiler_1'] = wood_pellets_1[i].varValue
    results.loc[i, 'wood_pellets_boiler_2'] = wood_pellets_2[i].varValue

# Print the results
#print(results)

results.to_excel("results.xlsx", index=False)

/Users/raul/opt/anaconda3/lib/python3.9/site-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/raul/opt/anaconda3/lib/python3.9/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/9k/hgnlw0z55v183zl9_r5xj9w40000gn/T/68a037d842ec48d99ac3395e508bae90-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/9k/hgnlw0z55v183zl9_r5xj9w40000gn/T/68a037d842ec48d99ac3395e508bae90-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 725 COLUMNS
At line 6774 RHS
At line 7495 BOUNDS
At line 7592 ENDATA
Problem MODEL has 720 rows, 432 columns and 5712 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 141 (-579) rows, 323 (-109) columns and 5015 (-697) elements
Perturbing problem by 0.001% of 1548.6 - largest nonzero change 0.00017053445 ( 1.8383649e-05%) - largest zero change 0.00017027658
0  Obj 0 Primal inf 1090.8114 (48)
47  Obj 178924.21 Primal inf 860.96282 (56)
98  Obj 222640.06 

In [ ]:
results

In [ ]:
heat_demand*(1+Network_losses)